In [22]:
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import time
import pandas as pd
from time import sleep
from tqdm import tqdm


creds = None
with open('../../secret.json') as creds_file:
    creds = json.load(creds_file)


# Set up the Spotify client
client_credentials_manager = SpotifyClientCredentials(client_id=creds['client_id'], client_secret=creds['client_secret'])
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

print('hi')
print(sp)


hi


In [23]:
# import time
# 
# artist_name = "Linkin Park"
# song_name = "Breaking the Habit"
# 
# try:
#     result = sp.search(q=f'track:{song_name} artist:{artist_name}', type='track')
#     print(result)
# except spotipy.SpotifyException as e:
#     print(f"An error occurred: {e}")
# finally:
#     # Introduce a delay between requests (e.g., 1 second)
#     time.sleep(1)


In [3]:
def get_song_release_year(song_name, artist_name):
    sleep(.5)
    # Search for the song
    # trying this search for faster queries hopefully
    # search(q, limit=10, offset=0, type='track', market=None)
    # let q = String.init(format:"artist:%@ track:%@",artistName,trackName)
    # https://stackoverflow.com/questions/42766000/spotipy-how-do-i-search-by-both-artist-and-song
    q = "artist:{} track:{}".format(artist_name, song_name)
    results = sp.search(q, limit=1, offset=0, type='track', market=None)

    # Extract the release year from the first search result
    if len(results['tracks']['items']) > 0:
        release_date = results['tracks']['items'][0]['album']['release_date']
        release_year = release_date.split('-')[0]
        return release_year
    else:
        # Return None if no results were found
        return None

In [6]:
artist_name = "Linkin Park"
song_name = "Breaking the Habit"
try:
    print(get_song_release_year(song_name, artist_name))
except Exception as e:
    print(str(e))



In [17]:
cddb_df = pd.read_csv('../../cleaning_stages/3_category_genre/cddb_2.tsv', sep='\t')
tracks_df = pd.read_csv('../../cleaning_stages/3_category_genre/cddb_tracks.tsv', sep='\t')

# not updating tracks here so can save to next cleaning stage
tracks_df.to_csv('../../cleaning_stages/4_year_repair/cddb_tracks.tsv', sep="\t", index=False)

cddb_df_trim = cddb_df[cddb_df['year'].isnull()]
cddb_df_trim = cddb_df_trim[['artist', 'tracks', 'year', 'id']]

In [18]:
# lists are much faster than dataframes, converting
cddb_list = cddb_df_trim.values.tolist()
tracks_list = tracks_df.values.tolist()

print(f'num null years: ${len(cddb_list)}')

num null years: $4532


In [19]:
# list should have 
n = 100
   
# using list comprehension 
cddb_list_chunks = [cddb_list[i:i + n] for i in range(0, len(cddb_list), n)] 
print(len(cddb_list_chunks))

46


In [20]:
def start_repair(cd_list, tracks_list):
    num_repairs = 0
    num_not_repairable = 0
    
    total_rows = len(cd_list)
    for cddb_row in tqdm(cd_list, total=total_rows, desc='Processing rows', ncols=100):
    #for cddb_row in cd_list:
  
        artist_name = cddb_row[0]
        album_id = cddb_row[1]

        # Grab the tracks for this album
        tracks_for_album = []
        for row in tracks_list:
            if row[1] == album_id:
                tracks_for_album.append(row[2])
            # stop iterating if album id is greater
            elif row[1] > int(album_id):
                break
                
        # tracks_for_album = tracks_df[tracks_df['Album Id'] == cddb_row['tracks']]
        
        if len(tracks_for_album) == 0:
            # No song for track something is wrong
            # print('TOSS: ' + str(cddb_row['id']) + ' ---- ' + artist_name)
            num_not_repairable += 1
            cd_list.remove(cddb_row)
            continue
        elif len(tracks_for_album) >= 5:
            # if first 5 songs fail then odds are the whole row fill fail
            tracks_for_album = tracks_for_album[:5]
        
        # old code
        #for _, track_row in enumerate(tracks_for_album):
        #    song_name = track_row['Name']
        #    fix = get_song_release_year(song_name, artist_name)
        #    if fix:
        #        # print(f'{artist_name} ---- {song_name} ---- {fix}')
        #        cddb_row['year'] = fix
        #        num_repairs += 1
        #        break # we found one we are clean
        
        for track in tracks_for_album:
            fix = get_song_release_year(track, artist_name)
            if fix:
                # print(f'{artist_name} ---- {song_name} ---- {fix}')
                # cddb_row[2] = fix
                num_repairs += 1
                # update cddb_df directly
                idx_list = cddb_df.index[cddb_df['id'] == cddb_row[3]].tolist()
                idx = idx_list[0]
                cddb_df.at[idx,'year'] = fix
                break # we found one we are clean
        
    print(f'Done! num repairs: {num_repairs}, num not repairable: {num_not_repairable}')
    return cd_list
    

In [21]:
# attemp to write output every 100 fixes 
itr = 0
for chunk in cddb_list_chunks:
    chunk = start_repair(chunk, tracks_list)
    print("finished chunk: " + str(itr))
    print("repaired songs: " + chunk)
    cddb_df.to_csv('../../cleaning_stages/4_year_repair/cddb_3.tsv', sep="\t", index=False)
    itr = itr + 1
   

Processing rows: 100%|████████████████████████████████████████████| 100/100 [03:30<00:00,  2.10s/it]


Done! num repairs: 40, num not repairable: 0
finished chunk: 0
[['Backstreet Boys', 1000, nan, 10000], ['Various', 1001, nan, 100001], ['No Return', 1002, nan, 100002], ['Ã¥å\xa0â€¡ã¥â€ºâ£ã¥â€ºâ€ºã¥â\xadâ£', 1005, nan, 100005], ['Luv Lite Massive', 1006, nan, 100006], ['Various', 1010, nan, 100010], ['Â¾ã®â¸â°ã€ãœ Worship', 1011, nan, 100011], ['Rumania Montevideo', 1012, nan, 100012], ['Keedy', 1014, nan, 100015], ['????', 1015, nan, 100016], ['Nature Destroyed', 1018, nan, 100020], ['Dingdong Avanzado', 1020, nan, 100022], ['Variousce Italiennes (disc 1)', 1022, nan, 100025], ['Bernadã¨te Bidã\xa0ude', 1026, nan, 100029], ['<v8.5e', 1029, nan, 100032], ['Gazebo', 1031, nan, 100035], ['Sportfreunde Stiller', 1033, nan, 100037], ['Rennie Pilgrim Presents', 1034, nan, 100038], ['Various', 1035, nan, 100039], ['Italo House', 1037, nan, 100041], ['Deux', 1038, nan, 100042], ['Various', 1040, nan, 100044], ['Robotika', 1043, nan, 100047], ['Johnny Hallyday', 1045, nan, 100049], ['Ã¤ã®ã«ã¥ã

Processing rows: 100%|████████████████████████████████████████████| 100/100 [03:11<00:00,  1.92s/it]


Done! num repairs: 50, num not repairable: 0
finished chunk: 1
[['Men Of Worth', 1205, nan, 100213], ['Betty Elders', 1206, nan, 100214], ['Wild Magnolias', 1207, nan, 100215], ['Amakye Dede', 1209, nan, 100217], ['Various Artists', 1210, nan, 100218], ['Joyas Del Mariachi', 1214, nan, 100223], ['Edith Piaf', 1215, nan, 100224], ['Munter Macher', 1217, nan, 100226], ['Carlos Di Sarli', 1222, nan, 100231], ['Carlos Nunez', 1224, nan, 100233], ['Harald Immig', 1225, nan, 100234], ['The Travelling Jewish Wedding', 1226, nan, 100235], ['One', 1228, nan, 100237], ['Mick Moloney', 1229, nan, 100238], ['Melanie', 1230, nan, 100239], ['Titti Bianchi & Ruggero Scandiuzzi', 1233, nan, 100242], ['Jari Sillanpã¤ã¤', 1235, nan, 100244], ['Bluatschink', 1237, nan, 100246], ['Ibrahim Tatlises', 1238, nan, 100247], ['Wilbury Clan', 1239, nan, 100248], ['Chris Newman And Mã¡ire Nã\xad Chathasaigh', 1240, nan, 100249], ['Brothers Four', 1242, nan, 100251], ['Various Artists', 1244, nan, 100253], ['After

Processing rows: 100%|████████████████████████████████████████████| 100/100 [03:27<00:00,  2.08s/it]


Done! num repairs: 45, num not repairable: 0
finished chunk: 2
[['Kazantzidis Stelios', 1404, nan, 100417], ['Massamba', 1408, nan, 100421], ['Ã‚ã«ã\xa0ã¤ã¨ã¬ã¨ã° Ã‚ã»ã±ã®ã¶ãªã¨ã©', 1409, nan, 100422], ['Highland Weavers/going Home', 1411, nan, 100424], ['Rolando Boldrin', 1412, nan, 100425], ['Belchior', 1414, nan, 100427], ['Tony Conjugacion', 1416, nan, 100429], ['Goran Ivanovic & Fareed Haque', 1421, nan, 100434], ['Despoina Vandi', 1423, nan, 100436], ['David Gray', 1425, nan, 100438], ['Fatih Kisaparmak', 1426, nan, 100439], ['Kanda Bongo Man', 1428, nan, 100441], ['Ãžã«ã¥ã£ Ãšã®ã±ã²ã°ã®ã¢', 1433, nan, 100446], ['Â³â¯â«ã˜â¦~', 1435, nan, 100449], ['Zeleziar', 1436, nan, 100450], ['Various', 1438, nan, 100452], ['Vicente Fernandez', 1440, nan, 100454], ['Lucã\xada De La Cruz', 1441, nan, 100455], ['Palemiger Spatzen', 1443, nan, 100457], ['Khamisu Khan', 1444, nan, 100458], ['Commonbond', 1445, nan, 100459], ['Kevn Kinney', 1446, nan, 100460], ['Various', 1447, nan, 100461], ['Vuj

Processing rows: 100%|████████████████████████████████████████████| 100/100 [03:12<00:00,  1.93s/it]


Done! num repairs: 50, num not repairable: 0
finished chunk: 3
[['Carlos Nãºã±ez', 1615, nan, 100634], ['Diary Of Dreams', 1619, nan, 100639], ['Peter White', 1620, nan, 10064], ['Flatt & Scruggs', 1623, nan, 100642], ['Various Artists', 1625, nan, 100644], ['Big Timber Bluegrass', 1626, nan, 100645], ['Varios', 1629, nan, 100648], ['Austria 7', 1631, nan, 100650], ['Le Mystã¨re Des Voix Bulgares', 1632, nan, 100651], ['Sing & Learn Vol1', 1634, nan, 100653], ['Kjarkas', 1635, nan, 100654], ['Various Artists', 1636, nan, 100655], ['German Hofmann', 1637, nan, 100656], ['Mama La Terre', 1638, nan, 100657], ['Lechner Buam', 1641, nan, 100660], ['Ann Reed', 1643, nan, 100662], ['Basia Frydman', 1644, nan, 100663], ['Lena Teigen', 1645, nan, 100664], ['David Francey', 1647, nan, 100666], ['Terje Tysland', 1648, nan, 100667], ['Schrammel & Slide', 1649, nan, 100668], ['Horst Chmela', 1650, nan, 100669], ['Moving Cloud', 1654, nan, 100673], ['Florie Brown', 1655, nan, 100674], ['Mexicanto', 

Processing rows:  61%|███████████████████████████▍                 | 61/100 [01:38<01:03,  1.62s/it]


KeyboardInterrupt: 

In [ ]:
# cddb_df[['artist', 'tracks', 'year']]
# cddb_df[[0, 1, 2]]

# tracks_df[['id', 'album id', 'name']]
# tracks_df[[0, 1, 2]]

# data.at[6,'NAME']='Safa'
# df.index[df['BoolCol']].tolist()